# Fitting the Australian Data with LRMoE

## Introduction

In this notebook, we will fit the Australian automobile data `ausprivauto0405` available in the `CASdatasets` [R package](http://cas.uqam.ca/).
We compare the performance of LRMoE mixture of Poisson distributions with the standard Poisson Generalized Linear Model (GLM).

In [ ]:
using DrWatson
@quickactivate "LRMoEjl Demo"

using CategoricalArrays, DataFrames, Distributions
using GLM, LRMoE, JLD2, PrettyTables, Random, StatsPlots

# some helper functions are hidden in a separate source file
include(srcdir("2023-CAS-Australian-util.jl"))
using .australian_auto_util_jl:
    load_aus_auto_jld,
    generate_aus_auto_LRMoE_data,
    predict_aus_auto_glm_distribution,
    predict_aus_auto_LRMoE_distribution,
    generate_pmf_comparison,
    plot_pmf_comparison

An overview of the dataset is given below.

In [ ]:
df = load_aus_auto_jld()
println("Number of observations: $(nrow(df))")
println("Sample rows of the data:")
pretty_table(first(df, 10))

In [ ]:
println("Distribution of claim frequency:")
df_claim_summary = combine(groupby(df, :ClaimNb),
    :ClaimNb => length => :ClaimNb_Count)
df_claim_summary.ClaimNb_Freq = df_claim_summary.ClaimNb_Count ./ sum(df_claim_summary.ClaimNb_Count)
pretty_table(df_claim_summary)

## Benchmark: Poisson GLM

We consider the Poisson GLM with the following covariates: `Gender`, `DrivAge`, `VehAge` and `VehValue`, with
the logarithm of `Exposure` also incorporated to predict the claim frequency `ClaimNb`. 

In [ ]:
fml_ClaimNb = @formula(ClaimNb ~  Gender + DrivAge + VehAge + VehValue)
glm_model = glm(fml_ClaimNb, df, Poisson(), LogLink(); offset=log.(df.Exposure))

In [ ]:
println("Fitted Loglikelihood: $(GLM.loglikelihood(glm_model))")
println("AIC: $(GLM.aic(glm_model))")
println("BIC: $(GLM.bic(glm_model))")

## LRMoE with Poisson Experts

We also consider fitting a 3-component LRMoE mixture of Poisson distributions with the same covariates as above.
The first step is to convert the original dataframe in to matrix formats
(support for the `@formula` interface is a feature under development).

In [ ]:
# convert data to matrix format for model fitting
y, X, y_col, X_col = generate_aus_auto_LRMoE_data(fml_ClaimNb, df)
exposure = df.Exposure
# view the converted data
println("First row of y: $(y[1, :])")
println("First row of X: $(X[1, :])")
println("Column names of X: $(X_col)")

The `LRMoE.jl` package provides a function for initializing a model, wihch provides initial
values `α_init` for the logit regression parameters and `params_init` for all possible expert functions.
We can pick out the Poisson initializations.

In [ ]:
# Random.seed!(20230315)
Random.seed!(42)
n_comp = 3
model_init = cmm_init(y, X, n_comp, ["discrete"]; exact_Y = true, n_random = 0)

In [ ]:
# pickout desired parameter initializations
α_init = model_init.α_init
experts_init = vcat([hcat([model_init.params_init[1][j][1] for j in 1:n_comp]...) for d in 1:1]...)
# view
println("α_init: $(α_init)")
println("experts_init: $(experts_init[1, :])")

The model initialization provided by the default function typically performs well.
Sometimes, we may also want to provide our own initializations given some domain knowledge and prior beliefs.
For example, if we believe the portfolio consists of high-, mid- and low-risk subgroups, the experts can be initialized as follows.

In [ ]:
experts_init_customized = [PoissonExpert(0.30);; PoissonExpert(0.10);; PoissonExpert(0.05)]

There are several settings to control the fitting function. For example, `ϵ` controls when to stop the EM algorithm based on the increment in loglikelihood, while `ecm_iter_max` provides a hard stop after a certain number of iterations. More details can be found in the package [documentation](https://actsci.utstat.utoronto.ca/LRMoE.jl/stable/fit/).

In [ ]:
LRMoE_model = fit_LRMoE(y, X, α_init, experts_init_customized;
    exposure=exposure, exact_Y=true, ϵ=0.01, ecm_iter_max=1000)

In [ ]:
summary(LRMoE_model)

The fitted LRMoE model shows better loglikelihood and AIC values than the Poisson GLM. It can be saved in the `JLD2` format for further analysis (see also the documentations [here](https://juliaio.github.io/JLD2.jl/stable/)).

In [ ]:
# save a fitted model
jldsave(datadir("2023-CAS-demo", "aus-auto-LRMoE-model.jld2"); model=LRMoE_model)
# load a fitted model
# LRMoE_model = load(datadir("2023-CAS-demo", "aus-auto-LRMoE-model.jld2"))["model"]

## Further Insights

### Comparison of Fitted Distributions

We see that LRMoE performs better when fitting the data, but exactly how?
This can be investigated by looking at the fitted distributions of Poisson GLM and LRMoE, and comparing then with the actual data.

In [ ]:
fitted_dist_glm = predict_aus_auto_glm_distribution(glm_model, df)
pretty_table(fitted_dist_glm)

In [ ]:
fitted_pmf_LRMoE = predict_aus_auto_LRMoE_distribution(LRMoE_model.model_fit, X, exposure)
pretty_table(fitted_pmf_LRMoE)

In [ ]:
# join empirical distribution and fitted distributions
# calculate the percentage error
df_pmf_comparison = generate_pmf_comparison(df_claim_summary, fitted_dist_glm, fitted_pmf_LRMoE)
pretty_table(df_pmf_comparison)

From the table above, we see that the Poisson GLM fails to capture the tail of the frequency distribution, severely underfitting
the probability mass function at 3 and 4+ claims. In contrast, LRMoE is able to capture the entirety of the frequency distribution
reasonably well. This can also be visualized by the following plot.

In [ ]:
plot_pmf_comparison(df_pmf_comparison)

We can also consider different ways of slicing the portfilio, e.g. by driver's age.
The LRMoE model outperforms in each group of drivers with similar ages, especially on the probability of 0, 1, and 2 claims.
The fitting result of LRMoE could be further imptoved if we consider an even larger number of latent classes.

In [ ]:
for driver_age in levels(df.DrivAge)[1:2]
    idx = (df.DrivAge .== driver_age)
    df_claim_summary_sub = combine(groupby(df[idx, :], :ClaimNb),
        :ClaimNb => length => :ClaimNb_Count)
    df_claim_summary_sub.ClaimNb_Freq = df_claim_summary_sub.ClaimNb_Count ./ sum(df_claim_summary_sub.ClaimNb_Count)
    fitted_dist_glm_sub = predict_aus_auto_glm_distribution(glm_model, df[idx, :])
    fitted_pmf_LRMoE_sub = predict_aus_auto_LRMoE_distribution(LRMoE_model.model_fit, X[idx, :], exposure[idx])
    df_pmf_comparison_sub = generate_pmf_comparison(df_claim_summary_sub, fitted_dist_glm, fitted_pmf_LRMoE_sub)
    println("Driver age: $driver_age, Number of observations: $(sum(idx))")
    pretty_table(df_pmf_comparison_sub)
    println("")
end

In [ ]:
for driver_age in levels(df.DrivAge)[3:4]
    idx = (df.DrivAge .== driver_age)
    df_claim_summary_sub = combine(groupby(df[idx, :], :ClaimNb),
        :ClaimNb => length => :ClaimNb_Count)
    df_claim_summary_sub.ClaimNb_Freq = df_claim_summary_sub.ClaimNb_Count ./ sum(df_claim_summary_sub.ClaimNb_Count)
    fitted_dist_glm_sub = predict_aus_auto_glm_distribution(glm_model, df[idx, :])
    fitted_pmf_LRMoE_sub = predict_aus_auto_LRMoE_distribution(LRMoE_model.model_fit, X[idx, :], exposure[idx])
    df_pmf_comparison_sub = generate_pmf_comparison(df_claim_summary_sub, fitted_dist_glm, fitted_pmf_LRMoE_sub)
    println("Driver age: $driver_age, Number of observations: $(sum(idx))")
    pretty_table(df_pmf_comparison_sub)
    println("")
end

In [ ]:
for driver_age in levels(df.DrivAge)[5:6]
    idx = (df.DrivAge .== driver_age)
    df_claim_summary_sub = combine(groupby(df[idx, :], :ClaimNb),
        :ClaimNb => length => :ClaimNb_Count)
    df_claim_summary_sub.ClaimNb_Freq = df_claim_summary_sub.ClaimNb_Count ./ sum(df_claim_summary_sub.ClaimNb_Count)
    fitted_dist_glm_sub = predict_aus_auto_glm_distribution(glm_model, df[idx, :])
    fitted_pmf_LRMoE_sub = predict_aus_auto_LRMoE_distribution(LRMoE_model.model_fit, X[idx, :], exposure[idx])
    df_pmf_comparison_sub = generate_pmf_comparison(df_claim_summary_sub, fitted_dist_glm, fitted_pmf_LRMoE_sub)
    println("Driver age: $driver_age, Number of observations: $(sum(idx))")
    pretty_table(df_pmf_comparison_sub)
    println("")
end

### Latent Groups of Policyholders

From the fitted expert functions, we see indeed that policyholders have different levels of risks as measured by the expected number of claims (i.e. the parameter of Poisson).

In [ ]:
dump(LRMoE_model.model_fit.comp_dist)

Based on the modelling structure of LRMoE, the covariates will affect the probabilities of each latent risk groups. Let us look at `Gender` as an example.

In [ ]:
# predict latent class probabilities by gender
idx = (df.Gender .== "Female")
latent_class_probs_female = predict_class_prior(X[idx, :], LRMoE_model.model_fit.α).prob
idx = (df.Gender .== "Male")
latent_class_probs_male = predict_class_prior(X[idx, :], LRMoE_model.model_fit.α).prob

# plot latent class probabilities
for j in [1, 2, 3]
    fig = plot(size=(750, 500))
    plot_min = minimum(vcat(latent_class_probs_male[:, j], latent_class_probs_female[:, j]))
    plot_max = maximum(vcat(latent_class_probs_male[:, j], latent_class_probs_female[:, j]))
    histogram!(latent_class_probs_male[:, j], label="Male", bins=(plot_min:0.005:plot_max), alpha=0.5, normalize=true)
    histogram!(latent_class_probs_female[:, j], label="Female", bins=(plot_min:0.005:plot_max), alpha=0.5, normalize=true)
    title!("Latent Class $j")
    display(fig)
end

While both genders share a similar distribution on the probability of Latent Class 1, female drivers tend to be more likely to belong to Latent Class 3, compared with male drivers. This in turn yields a higher prediction of the expected claim frequencies per year.

In [ ]:
idx = (df.Gender .== "Female")
predicted_mean_female = predict_mean_prior(X[idx, :], LRMoE_model.model_fit.α, LRMoE_model.model_fit.comp_dist)
idx = (df.Gender .== "Male")
predicted_mean_male = predict_mean_prior(X[idx, :], LRMoE_model.model_fit.α, LRMoE_model.model_fit.comp_dist)

fig = plot(size=(750, 500))
histogram!(predicted_mean_male[:, 1], label="Male", bins=0.10:0.001:0.25, alpha=0.5, normalize=true)
histogram!(predicted_mean_female[:, 1], label="Female", bins=0.10:0.001:0.25, alpha=0.5, normalize=true)
title!("Predicted Mean Frequency")
display(fig)

The discrepancy between male and female drivers are also reflected empirically in the actual data, which is captured by the LRMoE model.

In [ ]:
# Female
idx = (df.Gender .== "Female")
mean_frequency_empirical = sum(df[idx, "ClaimNb"]) / sum(df[idx, "Exposure"])
mean_frequency_GLM = mean(predict(glm_model, df[idx, :], offset=zero(log.(df[idx, "Exposure"]))))
mean_frequency_LRMoE = mean(predicted_mean_female)
println("Female Drivers:")
println("Empirical mean frequency: $mean_frequency_empirical")
println("GLM predicted mean frequency: $mean_frequency_GLM, ($((mean_frequency_GLM-mean_frequency_empirical)/mean_frequency_empirical *100)%)")
println("LRMoE predicted mean frequency: $mean_frequency_LRMoE, ($((mean_frequency_LRMoE-mean_frequency_empirical)/mean_frequency_empirical *100)%)")
# Male
idx = (df.Gender .== "Male")
mean_frequency_empirical = sum(df[idx, "ClaimNb"]) / sum(df[idx, "Exposure"])
mean_frequency_GLM = mean(predict(glm_model, df[idx, :], offset=zero(log.(df[idx, "Exposure"]))))
mean_frequency_LRMoE = mean(predicted_mean_male)
println("Male Drivers:")
println("Empirical mean frequency: $mean_frequency_empirical")
println("GLM predicted mean frequency: $mean_frequency_GLM, ($((mean_frequency_GLM-mean_frequency_empirical)/mean_frequency_empirical *100)%)")
println("LRMoE predicted mean frequency: $mean_frequency_LRMoE, ($((mean_frequency_LRMoE-mean_frequency_empirical)/mean_frequency_empirical *100)%)")